In [1]:
!pip install simpletransformers==0.60.9

In [2]:
import torch
use_cuda = torch.cuda.is_available()
print(f"CUDA available: {use_cuda}")

CUDA available: False


In [3]:
from simpletransformers.seq2seq import Seq2SeqModel, Seq2SeqArgs
import pandas as pd
import logging
from sklearn.model_selection import train_test_split
import os
from datetime import datetime
import warnings
print("done")

done


define loading function 

In [4]:

def load_data(file_path, input_text_column, target_text_column, label_column, keep_label=1):
    try:
        df = pd.read_csv(file_path, sep='\t', on_bad_lines='skip')
    except pd.errors.ParserError:
        df = pd.read_csv(file_path, sep=',', on_bad_lines='skip')
    df = df[df[label_column] == keep_label]
    df = df.rename(columns={input_text_column: 'input_text', target_text_column: 'target_text'})
    df = df[['input_text', 'target_text']]
    return df
print("done")

done


In [5]:

#cleaning function 
def clean_unnecessary_spaces(out_string):
    if not isinstance(out_string, str):
        # Convert non-string types to string without issuing a warning
        out_string = str(out_string)

    # Clean unnecessary spaces in the string
    out_string = (
        out_string.replace(".",".")
        .replace(" ?","?")
        .replace(" !","!")
        .replace(" ,",",")
        .replace(" ' ","'")
        .replace(" n't","n't")
        .replace(" 's","'s")
        .replace(" 'm","'m")
        .replace(" 're","'re")
        .replace(" 've","'ve")
        .replace(" 'll","'ll")
    )
    return out_string
print("done")

done


In [6]:
#Google Data
train_df = pd.read_csv("train (1).tsv", sep="\t").astype(str)
eval_df = pd.read_csv("dev (1).tsv", sep="\t").astype(str)

train_df = train_df.loc[train_df["label"] == "1"]
eval_df = eval_df.loc[eval_df["label"] == "1"]

train_df = train_df.rename(
    columns={"sentence1": "input_text", "sentence2": "target_text"}
)
eval_df = eval_df.rename(
    columns={"sentence1": "input_text", "sentence2": "target_text"}
)

train_df = train_df[["input_text", "target_text"]]
eval_df = eval_df[["input_text", "target_text"]]

train_df["prefix"] = "paraphrase"
eval_df["prefix"] = "paraphrase"

print(train_df)
print("---------")
print(eval_df)


                                              input_text  \
1      The NBA season of 1975 -- 76 was the 30th seas...   
3      When comparable rates of flow can be maintaine...   
4      It is the seat of Zerendi District in Akmola R...   
5      William Henry Henry Harman was born on 17 Febr...   
7      With a discrete amount of probabilities Formul...   
...                                                  ...   
49384  The Romanesque language , Galician ( Galego ) ...   
49390  Note that k is a vector consisting of three in...   
49393  Tim Henman won in the final 6 -- 2 , 7 -- 6 , ...   
49395  He was considered an active member of the coun...   
49397  She was in Cork on June 24 and arrived on 8 Ju...   

                                             target_text      prefix  
1      The 1975 -- 76 season of the National Basketba...  paraphrase  
3      The results are high when comparable flow rate...  paraphrase  
4      It is the seat of the district of Zerendi in A...  paraphra

In [7]:
#MSRP Data
train_df = pd.concat(
    [
        train_df,
        load_data("msr_paraphrase_train (1).txt", "#1 String", "#2 String", "Quality"),
    ]
)
eval_df = pd.concat(
    [
        eval_df,
        load_data("msr_paraphrase_test (1).txt", "#1 String", "#2 String", "Quality"),
    ]
)
print("done")

done


In [8]:
#Quora Data
# The Quora Dataset is not separated into train/test, so we do it manually the first time.
df = load_data(
    "quora_duplicate_questions (1).tsv", "question1", "question2", "is_duplicate"
)
q_train, q_test = train_test_split(df)
print("done")

done


In [9]:

#parabank dataset 5m
import warnings
import pandas as pd
from simpletransformers.seq2seq import Seq2SeqModel, Seq2SeqArgs
from sklearn.model_selection import train_test_split
warnings.filterwarnings("ignore", category=FutureWarning)
paradata = pd.read_csv("parabank_5m (1).tsv", sep='\t', header=None, on_bad_lines='skip')
paradata = paradata[0:100000]
paradata['prefix'] = 'paraphrase'
paradata.rename(columns={0: 'input_text', 1: 'target_text'}, inplace=True)
para_train, para_test = train_test_split(paradata)
train_df = pd.concat([train_df, q_train, para_train])
eval_df = pd.concat([eval_df, q_test, para_test])
train_df = train_df[["prefix", "input_text", "target_text"]].dropna()
eval_df = eval_df[["prefix", "input_text", "target_text"]].dropna()
train_df["input_text"] = train_df["input_text"].apply(clean_unnecessary_spaces)
train_df["target_text"] = train_df["target_text"].apply(clean_unnecessary_spaces)
eval_df["input_text"] = eval_df["input_text"].apply(clean_unnecessary_spaces)
eval_df["target_text"] = eval_df["target_text"].apply(clean_unnecessary_spaces)
model_args = Seq2SeqArgs()
model_args.do_sample = True
model_args.eval_batch_size = 32
model_args.evaluate_during_training = True
model_args.evaluate_during_training_steps = 1000
model_args.evaluate_during_training_verbose = True
model_args.fp16 = False
model_args.learning_rate = 5e-5
model_args.max_length = 64
model_args.max_seq_length = 64
model_args.num_beams = None
model_args.num_return_sequences = 3
model_args.num_train_epochs = 1
model_args.overwrite_output_dir = True
model_args.reprocess_input_data = True
model_args.save_eval_checkpoints = True
model_args.save_steps = 2000
model_args.evaluate_during_training_verbose = True
model_args.top_k = 50
model_args.top_p = 0.95
model_args.train_batch_size = 16
model_args.use_multiprocessing = False
model_args.wandb_project = None
model = Seq2SeqModel(
    encoder_decoder_type="bart",
    encoder_decoder_name=r"outputs\best_model",
    args=model_args,
    use_cuda=False
)
model.train_model(train_df, eval_data=eval_df, verbose=True)
to_predict = [
    prefix + ": " + str(input_text)
    for prefix, input_text in zip(eval_df["prefix"].tolist(), eval_df["input_text"].tolist())
]
truth = eval_df["target_text"].tolist()
preds = model.predict(to_predict)
print("done")

  0%|          | 0/96270 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/6017 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [1]:
from simpletransformers.seq2seq import Seq2SeqModel, Seq2SeqArgs

# Load the fine-tuned BART model for paraphrasing (checkpoint-6020-epoch-1)
model_args = Seq2SeqArgs()
model_args.do_sample = True
model_args.eval_batch_size = 32
model_args.max_length = 64
model_args.num_return_sequences = 1  # Default number of outputs for paraphrasing

paraphrase_model = Seq2SeqModel(
    encoder_decoder_type="bart",
    encoder_decoder_name=r"outputs\best_model",  # Updated path to fine-tuned model
    args=model_args,
    use_cuda=False
)

# Function for paraphrasing
def paraphrase_text(input_text, num_return_sequences=1):
    model_args.num_return_sequences = num_return_sequences
    to_predict = [f"paraphrase: {input_text.strip()}"]
    predictions = paraphrase_model.predict(to_predict)
    
    # Print predictions to debug
    print("Predictions:", predictions)
    
    # Flatten predictions if it's a list of lists
    if isinstance(predictions[0], list):
        flattened_predictions = [p for sublist in predictions for p in sublist]  # Flattening the list of lists
        return clean_paraphrase_output(flattened_predictions)  # Clean the flattened predictions
    
    return clean_paraphrase_output(predictions) if predictions else ["No output generated."]

# Function to clean paraphrase output
# Function to clean paraphrase output
def clean_paraphrase_output(predictions):
    cleaned_output = []
    for p in predictions:
        if isinstance(p, str):
            # Remove common prefixes and extra spaces
            cleaned_p = (
                p.replace("aphrase:", "")
                .replace("aphent:", "")
                .replace("aphdomenas:", "")
                .replace("aphdomenas :", "")
                .replace("aphrase :", "")
                .replace("aphrase :", "")
                .replace(" aphrase:", "")
                .replace(" aphrase :", "")
                .replace("  aphrase :", "")
                .replace("  aphrase:", "")
                .replace(" aphrase : ", "")
                .strip()
            )
            cleaned_output.append(cleaned_p)
    return cleaned_output


# Example input
input_text = "he is a good player "
num_paraphrases = 6  

# Get paraphrased results
result = paraphrase_text(input_text, num_return_sequences=num_paraphrases)

# Display the results
for idx, paraphrase in enumerate(result):
    print(f"Paraphrase {idx + 1}: {paraphrase}")


Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

c:\Users\athar\anaconda3\Lib\site-packages\transformers\generation\configuration_utils.py:615: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
c:\Users\athar\anaconda3\Lib\site-packages\transformers\generation\configuration_utils.py:634: UserWarning: `num_beams` is set to 1. However, `length_penalty` is set to `2.0` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `length_penalty`.
  warnings.warn(


Predictions: [['paraphrase: he is a good player', 'paraphrase: Do it he is a good player', 'paraphrase: he is a good player', 'paraphrase: he is a good player home defence player I am a good...', 'paraphrase: he is a good player', 'paraphrase: he is a good operator and a good player']]
Paraphrase 1: par he is a good player
Paraphrase 2: par Do it he is a good player
Paraphrase 3: par he is a good player
Paraphrase 4: par he is a good player home defence player I am a good...
Paraphrase 5: par he is a good player
Paraphrase 6: par he is a good operator and a good player


In [11]:
print(train_df.head())
print(train_df.columns)
print(eval_df.head())
print(eval_df.columns)

       prefix                                         input_text  \
1  paraphrase  The NBA season of 1975 -- 76 was the 30th seas...   
3  paraphrase  When comparable rates of flow can be maintaine...   
4  paraphrase  It is the seat of Zerendi District in Akmola R...   
5  paraphrase  William Henry Henry Harman was born on 17 Febr...   
7  paraphrase  With a discrete amount of probabilities Formul...   

                                         target_text  
1  The 1975 -- 76 season of the National Basketba...  
3  The results are high when comparable flow rate...  
4  It is the seat of the district of Zerendi in A...  
5  William Henry Harman was born in Waynesboro, V...  
7  Given a discrete set of probabilities formula ...  
Index(['prefix', 'input_text', 'target_text'], dtype='object')
       prefix                                         input_text  \
1  paraphrase  They were there to enjoy us and they were ther...   
2  paraphrase  After the end of the war in June 1902, Higgins.

In [12]:
print(train_df.dtypes)
print(eval_df.dtypes)

prefix         object
input_text     object
target_text    object
dtype: object
prefix         object
input_text     object
target_text    object
dtype: object


In [13]:
print("Training Data Missing Values:\n", train_df.isnull().sum())
print("Evaluation Data Missing Values:\n", eval_df.isnull().sum())

Training Data Missing Values:
 prefix         0
input_text     0
target_text    0
dtype: int64
Evaluation Data Missing Values:
 prefix         0
input_text     0
target_text    0
dtype: int64


In [14]:
import torch
import pandas as pd
from transformers import BartTokenizer, BartForConditionalGeneration
from nltk.translate.bleu_score import sentence_bleu
import nltk
nltk.download('punkt')  # Ensure that necessary NLTK data is downloaded

# Load your trained model and tokenizer
tokenizer = BartTokenizer.from_pretrained(r"outputs\best_model")
model = BartForConditionalGeneration.from_pretrained(r"outputs\best_model")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Function to generate paraphrases using the custom model
def generate_paraphrase(input_sentence):
    encoding = tokenizer.encode_plus(input_sentence, return_tensors="pt", padding=True)
    input_ids = encoding["input_ids"].to(device)
    attention_mask = encoding["attention_mask"].to(device)

    # Generate paraphrase
    output = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=128)
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Function to evaluate the model accuracy
def evaluate_model(tsv_file):
    # Load the TSV file into a pandas DataFrame
    eval_df = pd.read_csv(tsv_file, sep='\t')
    
    references = []  # Store the true paraphrases (sentence2)
    hypotheses = []  # Store the generated paraphrases (model output)
    
    for _, row in eval_df.iterrows():
        input_sentence = row['sentence1']  # Original sentence (input)
        true_paraphrase = row['sentence2']  # Expected paraphrase (target)
        
        # Generate paraphrase using the model
        generated_paraphrase = generate_paraphrase(input_sentence)
        
        # Tokenize the reference and hypothesis sentences
        reference_tokens = [nltk.word_tokenize(true_paraphrase)]
        hypothesis_tokens = nltk.word_tokenize(generated_paraphrase)
        
        references.append(reference_tokens)
        hypotheses.append(hypothesis_tokens)

    # Calculate BLEU score for each sentence
    bleu_scores = [sentence_bleu(ref, hyp) for ref, hyp in zip(references, hypotheses)]
    
    # Calculate the average BLEU score across all sentences
    avg_bleu_score = sum(bleu_scores) / len(bleu_scores)
    
    print(f"Average BLEU Score: {avg_bleu_score:.4f}")

# Path to your TSV file
tsv_file = "test (1).tsv"  # Relative path to the file

# Run the evaluation
evaluate_model(tsv_file)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\athar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


KeyboardInterrupt: 

In [ ]:
# !pip install simpletransformers==0.60.9
# import torch
# use_cuda = torch.cuda.is_available()
# print(f"CUDA available: {use_cuda}")





# from simpletransformers.seq2seq import Seq2SeqModel, Seq2SeqArgs
# import pandas as pd
# import logging
# from sklearn.model_selection import train_test_split
# import os
# from datetime import datetime
# import warnings
# print("done")
# def load_data(file_path, input_text_column, target_text_column, label_column, keep_label=1):
#     try:
#         # Load data, skipping bad lines
#         df = pd.read_csv(file_path, sep='\t', on_bad_lines='skip')
#     except pd.errors.ParserError:
#         # If the file uses a different delimiter, try with comma
#         df = pd.read_csv(file_path, sep=',', on_bad_lines='skip')
#     df = df[df[label_column] == keep_label]
#     df = df.rename(columns={input_text_column: 'input_text', target_text_column: 'target_text'})
#     df = df[['input_text', 'target_text']]

#     return df
# print("done")
# import warnings

# def clean_unnecessary_spaces(out_string):
#     if not isinstance(out_string, str):
#         # Convert non-string types to string without issuing a warning
#         out_string = str(out_string)

#     # Clean unnecessary spaces in the string
#     out_string = (
#         out_string.replace(".",".")
#         .replace(" ?","?")
#         .replace(" !","!")
#         .replace(" ,",",")
#         .replace(" ' ","'")
#         .replace(" n't","n't")
#         .replace(" 's","'s")
#         .replace(" 'm","'m")
#         .replace(" 're","'re")
#         .replace(" 've","'ve")
#         .replace(" 'll","'ll")
#     )
#     return out_string
# print("done")
# #Google Data
# train_df = pd.read_csv("train (1).tsv", sep="\t").astype(str)
# eval_df = pd.read_csv("dev (1).tsv", sep="\t").astype(str)

# train_df = train_df.loc[train_df["label"] == "1"]
# eval_df = eval_df.loc[eval_df["label"] == "1"]

# train_df = train_df.rename(
#     columns={"sentence1": "input_text", "sentence2": "target_text"}
# )
# eval_df = eval_df.rename(
#     columns={"sentence1": "input_text", "sentence2": "target_text"}
# )

# train_df = train_df[["input_text", "target_text"]]
# eval_df = eval_df[["input_text", "target_text"]]

# train_df["prefix"] = "paraphrase"
# eval_df["prefix"] = "paraphrase"

# print(train_df)
# print("---------")
# print(eval_df)

# #MSRP Data
# train_df = pd.concat(
#     [
#         train_df,
#         load_data("msr_paraphrase_train (1).txt", "#1 String", "#2 String", "Quality"),
#     ]
# )
# eval_df = pd.concat(
#     [
#         eval_df,
#         load_data("msr_paraphrase_test (1).txt", "#1 String", "#2 String", "Quality"),
#     ]
# )
# print("done")
# #Quora Data
# # The Quora Dataset is not separated into train/test, so we do it manually the first time.
# df = load_data(
#     "quora_duplicate_questions (1).tsv", "question1", "question2", "is_duplicate"
# )
# q_train, q_test = train_test_split(df)
# print("done")
# import warnings
# import pandas as pd
# from simpletransformers.seq2seq import Seq2SeqModel, Seq2SeqArgs
# from sklearn.model_selection import train_test_split
# warnings.filterwarnings("ignore", category=FutureWarning)
# paradata = pd.read_csv("parabank_5m (1).tsv", sep='\t', header=None, on_bad_lines='skip')
# paradata = paradata[0:100000]
# paradata['prefix'] = 'paraphrase'
# paradata.rename(columns={0: 'input_text', 1: 'target_text'}, inplace=True)
# para_train, para_test = train_test_split(paradata)
# train_df = pd.concat([train_df, q_train, para_train])
# eval_df = pd.concat([eval_df, q_test, para_test])
# train_df = train_df[["prefix", "input_text", "target_text"]].dropna()
# eval_df = eval_df[["prefix", "input_text", "target_text"]].dropna()
# train_df["input_text"] = train_df["input_text"].apply(clean_unnecessary_spaces)
# train_df["target_text"] = train_df["target_text"].apply(clean_unnecessary_spaces)
# eval_df["input_text"] = eval_df["input_text"].apply(clean_unnecessary_spaces)
# eval_df["target_text"] = eval_df["target_text"].apply(clean_unnecessary_spaces)
# model_args = Seq2SeqArgs()
# model_args.do_sample = True
# model_args.eval_batch_size = 32
# model_args.evaluate_during_training = True
# model_args.evaluate_during_training_steps = 1000
# model_args.evaluate_during_training_verbose = True
# model_args.fp16 = False
# model_args.learning_rate = 5e-5
# model_args.max_length = 64
# model_args.max_seq_length = 64
# model_args.num_beams = None
# model_args.num_return_sequences = 3
# model_args.num_train_epochs = 1
# model_args.overwrite_output_dir = True
# model_args.reprocess_input_data = True
# model_args.save_eval_checkpoints = True
# model_args.save_steps = 2000
# model_args.evaluate_during_training_verbose = True
# model_args.top_k = 50
# model_args.top_p = 0.95
# model_args.train_batch_size = 16
# model_args.use_multiprocessing = False
# model_args.wandb_project = None
# model = Seq2SeqModel(
#     encoder_decoder_type="bart",
#     encoder_decoder_name=r"outputs\checkpoint-6019-epoch-1",
#     args=model_args,
#     use_cuda=False
# )
# model.train_model(train_df, eval_data=eval_df, verbose=True)
# to_predict = [
#     prefix + ": " + str(input_text)
#     for prefix, input_text in zip(eval_df["prefix"].tolist(), eval_df["input_text"].tolist())
# ]
# truth = eval_df["target_text"].tolist()
# preds = model.predict(to_predict)
# print("done")
# import warnings
# import pandas as pd
# from simpletransformers.seq2seq import Seq2SeqModel, Seq2SeqArgs
# from sklearn.model_selection import train_test_split

# warnings.filterwarnings("ignore", category=FutureWarning)

# paradata = pd.read_csv("parabank_5m (1).tsv", sep='\t', header=None, on_bad_lines='skip')
# paradata = paradata[0:100000]
# paradata['prefix'] = 'paraphrase'
# paradata.rename(columns={0: 'input_text', 1: 'target_text'}, inplace=True)

# para_train, para_test = train_test_split(paradata)
# train_df = para_train[["prefix", "input_text", "target_text"]].dropna()
# eval_df = para_test[["prefix", "input_text", "target_text"]].dropna()

# def clean_unnecessary_spaces(text):
#     return " ".join(text.split())

# train_df["input_text"] = train_df["input_text"].apply(clean_unnecessary_spaces)
# train_df["target_text"] = train_df["target_text"].apply(clean_unnecessary_spaces)
# eval_df["input_text"] = eval_df["input_text"].apply(clean_unnecessary_spaces)
# eval_df["target_text"] = eval_df["target_text"].apply(clean_unnecessary_spaces)

# # Set up model arguments
# model_args = Seq2SeqArgs()
# model_args.do_sample = True
# model_args.eval_batch_size = 32
# model_args.max_length = 64
# model_args.num_return_sequences = 3
# model_args.num_beams = 5  # Set to a value > 1 for better generation
# model_args.use_multiprocessing = False

# model = Seq2SeqModel(
#     encoder_decoder_type="bart",
#     encoder_decoder_name=r"C:\Users\athar\Downloads\para_project\outputs\checkpoint-6019-epoch-1",
#     args=model_args,
#     use_cuda=False
# )

# input_sentences = [
#     "Chess is a board game for two players. It is sometimes called international chess or Western chess to distinguish it from related games such as xiangqi (Chinese chess) and shogi (Japanese chess)",
# ]

# to_predict = ["paraphrase: " + sentence for sentence in input_sentences]

# predictions = model.predict(to_predict)
# for input_sentence, pred in zip(input_sentences, predictions):
#     print(f"Input: {input_sentence}\nParaphrased:")
#     for p in pred:
#         print(f"  - {p.replace('paraphrase: ', '').strip()}")
#     print()  

# def generate_paraphrase(input_sentence):
#     to_predict = [f"paraphrase: {input_sentence}"]
#     paraphrase = model.predict(to_predict)
#     return paraphrase[0] if paraphrase else None
# input_sentence = "The chocolate cake was delicious and very enjoyable"
# paraphrased_sentence = generate_paraphrase(input_sentence)
# print("Input Sentence: ", input_sentence)
# print( paraphrased_sentence)
# print(train_df.head())
# print(train_df.columns)
# print(eval_df.head())
# print(eval_df.columns)
# print("Training Data Missing Values:\n", train_df.isnull().sum())
# print("Evaluation Data Missing Values:\n", eval_df.isnull().sum())
# import torchprint("Training Data Missing Values:\n", train_df.isnull().sum())
# print("Evaluation Data Missing Values:\n", eval_df.isnull().sum())
# import pandas as pd
# from transformers import BartTokenizer, BartForConditionalGeneration
# from nltk.translate.bleu_score import sentence_bleu
# import nltk
# nltk.download('punkt')  # Ensure that necessary NLTK data is downloaded

# # Load your trained model and tokenizer
# tokenizer = BartTokenizer.from_pretrained(r"outputs\checkpoint-6020-epoch-1")
# model = BartForConditionalGeneration.from_pretrained(r"outputs\checkpoint-6020-epoch-1")
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)

# # Function to generate paraphrases using the custom model
# def generate_paraphrase(input_sentence):
#     encoding = tokenizer.encode_plus(input_sentence, return_tensors="pt", padding=True)
#     input_ids = encoding["input_ids"].to(device)
#     attention_mask = encoding["attention_mask"].to(device)

#     # Generate paraphrase
#     output = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=128)
#     return tokenizer.decode(output[0], skip_special_tokens=True)

# # Function to evaluate the model accuracy
# def evaluate_model(tsv_file):
#     # Load the TSV file into a pandas DataFrame
#     eval_df = pd.read_csv(tsv_file, sep='\t')
    
#     references = []  # Store the true paraphrases (sentence2)
#     hypotheses = []  # Store the generated paraphrases (model output)
    
#     for _, row in eval_df.iterrows():
#         input_sentence = row['sentence1']  # Original sentence (input)
#         true_paraphrase = row['sentence2']  # Expected paraphrase (target)
        
#         # Generate paraphrase using the model
#         generated_paraphrase = generate_paraphrase(input_sentence)
        
#         # Tokenize the reference and hypothesis sentences
#         reference_tokens = [nltk.word_tokenize(true_paraphrase)]
#         hypothesis_tokens = nltk.word_tokenize(generated_paraphrase)
        
#         references.append(reference_tokens)
#         hypotheses.append(hypothesis_tokens)

#     # Calculate BLEU score for each sentence
#     bleu_scores = [sentence_bleu(ref, hyp) for ref, hyp in zip(references, hypotheses)]
    
#     # Calculate the average BLEU score across all sentences
#     avg_bleu_score = sum(bleu_scores) / len(bleu_scores)
    
#     print(f"Average BLEU Score: {avg_bleu_score:.4f}")

# # Path to your TSV file
# tsv_file = "test (1).tsv"  # Relative path to the file

# # Run the evaluation
# evaluate_model(tsv_file)
